# Qiskit Quantinuum Provider Example

This notebook contains an example of circuit submission via the Qiskit Quantinuum Provider.

You will need Quantinuum Hardware Systems access to run this notebook. The notebook will prompt you for your password or you can set it in your environment as `QUANTINUUM_API_KEY`.

## Set up Connection

Login to the Quantinuum API using your credentials, choose your device, and check the device status. For information on the devices available see the *Quantinuum Systems User Guide* in the *Examples* tab on the Quantinuum User Portal.

In [1]:
import numpy as np

from qiskit import QuantumCircuit, execute
from qiskit_quantinuum import Quantinuum
from qiskit.visualization import plot_histogram

Load the account credentials. You will be prompted for your password. 

*Windows Users:* If you get a `RequestsApiError` mentioning an `SSLCertVerificationError` install the following in your python environment, restart your kernel and try again: `pip install python-certifi-win32`.

In [2]:
# Quantinuum.save_account(user_name="first.last@company.com")
Quantinuum.save_account(user_name="megan.l.kohagen@quantinuum.com")
Quantinuum.load_account()

Enter your password: ········
***Successfully logged in***


Look at a list of the available Quantinuum devices.

In [ ]:
Quantinuum.backends()

Choose the device you'd like to work with and check its status.

In [4]:
device = 'H1-2E'
backend = Quantinuum.get_backend(device)
status = backend.status()

print(device, "status :", status.status_msg)

H1-2E status : online


## Circuit Preparation

The Quantinuum API accepts circuits in the [OpenQASM](https://github.com/Qiskit/openqasm) format. The benefit of using the Qiskit Quantinuum provider is that it supports conversion to and from the OpenQASM format before submission to a Quantinuum device. 

Note that qiskit may not have all additional capabilities enabled by the the Quantinuum OpenQASM Extension. See the *Extensions Example* notebook on the Quantinuum User Portal for information about Quantinuum-specific OpenQASM extensions.

Here we generate a Bell-state preparation circuit using qiskit's `QuantumCircuit` object.

In [5]:
n_qubits = 2
circuit = QuantumCircuit(n_qubits, n_qubits, name='Bell Test')
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

circuit.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
   c: 2/══════════════╬══╬═
                      ║  ║ 
meas: 2/══════════════╩══╩═
                      0  1

## Submit the Circuit

Now we submit the circuit with the `execute` function.

- `shots`: The number of times you want to run this quantum program

In [6]:
shots = 100
job = execute(circuit, backend, shots=shots)

## Check Job Status

Once submitted, a job's status can be checked using the `QuantinuumJob`'s `status` function. This will return information on whether the job has been queued or completed.

In [7]:
job.status()

<JobStatus.DONE: 'job has successfully run'>

## Return Results

When a job completes the results can be fetched using the `result()` function.

In [8]:
result = job.result()

print("Job ID:", result.job_id)
print("Job success:", result.success)

Job ID: f151861654594c8fb718541dede4476d
Job success: True


## Analyzing Results

The `get_counts` function returns the aggregate outcomes of the experiment you ran.

For the Bell Test we ran, the output bitstring should be `00` approximately 50 percent of the time. 

In [9]:
counts = result.get_counts()
print("Total count for 00 and 11 are:",counts)

# Result should not be '0', it should be '00'!

Total count for 00 and 11 are: {'11': 52, '0': 47, '1': 1}


Qiskit provides many visualizations, including the function `plot_histogram`, to view your results if `matplotlib` is installed.

In [ ]:
plot_histogram(counts)

## Saving Results

It is **highly** recommended to save results after jobs are completed in order to avoid losing results. The Quantinuum data retention policy is found in the *Quantinuum Systems User Guide* under the *Examples* tab on the Quantinuum User Portal. Below the results are saved in a json file.

In [ ]:
import json

with open(f'results_{result.job_id}.json', 'w') as f:
    json.dump(result.get_counts(), f)

## Results for Past Jobs

You may wish to return the results for a job you ran that already completed.

*Note:* When running multiple experiments, a `QOjb` object returns multiple `job_id`s. Currently, the only method of retrieving jobs is through an individual `job_id`. You can retrieve all `job_id`s from a job by running `job.job_ids()`

In [ ]:
device = 'H1-2E'
job_id = 'insert job id'
backend = Quantinuum.get_backend(device)
job = backend.retrieve_job(job_id)
result = job.result()
counts = result.get_counts()
print(counts)

## Submit to a Machine Family

If you would like faster access to hardware between the available quantum computers, you can submit to a Hardware Family. Submitting this way will run the job on whatever device is available first that has the number of qubits needed to run the circuit. 

In [ ]:
device = 'H1'
shots = 100

backend = Quantinuum.get_backend(device)
job = execute(circuit, backend, shots=shots)

result = job.result()
counts = result.get_counts()
print(counts)

## Recommended Workflow <a class="anchor" id="recommended-workflow"></a>

Quantinuum provides device-specific syntax checkers and emulators in addition to quantum computing hardware. Quantum algorithm development takes a lot of time to create a quantum circuit, run on hardware, analyze results, and iterate. Within this loop it is common to go through multiple debugging cycles either due to code or circuit design. 

A recommended workflow for working with the Quantinuum hardware is the following:
1. Syntax Checker
2. Emulator
3. Quantum Computer

The first step will allow you to check whether or not your circuit will run on the quantum computer, whether there are any code mistakes. The second step will allow you to run on a device-specific emulator and analyze results. This will allow you to check the output to determine if it is what you expect. This will allow you to debug your circuit design. The Syntax Checkers and Emulators are available 24/7, allowing you to debug your code as needed befor submitting to a quantum computer.

If you're using this Qiskit Quantinuum provider, step 1 should be taken care of for you since Qiskit handles the conversion of a qiskit `QuantumCircuit` object and OpenQASM. However, it may still be beneficial to test before using any H-System Quantum Credits. In addition, always submit to an emulator before submitting to a quantum computer. This will help you determine if any errors are occurring in your circuit design.

&copy; 2019-2022 Quantinuum. (www.quantinuum.com)

Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at:

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either  
express or implied. See the License for the specific language  
governing permissions and limitations under the License.